In [ ]:
#|hide
#|default_exp serve

# serve

> fastdaemon server

In [ ]:
#|export
import importlib
import sys
from contextlib import contextmanager
from functools import partial
from io import StringIO
from multiprocessing import get_context
from socketserver import TCPServer, StreamRequestHandler, ThreadingTCPServer, ThreadingUnixStreamServer

from fastcore.parallel import ProcessPoolExecutor
from fastcore.script import *

from fastdaemon.core import *

from datetime import datetime # TODO: remove after optimising

In [ ]:
#|hide
from nbprocess.showdoc import *

In [ ]:
import time
from fastcore.parallel import threaded
from pathlib import Path
from socketserver import UnixStreamServer

In [ ]:
#|export
def _setattrs(o, d):
    for k,v in d.items(): setattr(o,k,v)

In [ ]:
#|export
@contextmanager
def _redirect_streams(argv, stdin, stdout, stderr):
    new = {k:v for k,v in locals().items()}
    old = {o:getattr(sys,o) for o in new.keys()}
    _setattrs(sys, new)
    try: yield new['stdout'],new['stderr']
    finally: _setattrs(sys, old)

In [ ]:
#|export
def _run(cmd, argv, stdin, stdout, stderr):
    with _redirect_streams(argv,stdin,stdout,stderr): cmd()
    return stdout,stderr

In [ ]:
#|export
class CmdHandler(StreamRequestHandler):
    "Run `self.server.cmd` with request's `argv` and `stdin`; return `stdout` and `stderr`"
    def setup(self):
        super().setup()
        stdin,argv = recv_record(self.rfile.read)
        self.argv = [self.server.cmd.__name__] + argv.split(' ') if argv else []
        self.stdin,self.stdout,self.stderr = StringIO(stdin),StringIO(),StringIO()
        print(f'len(stdin)={len(self.stdin.getvalue())} argv={self.argv}')
    def finish(self):
        self.stdout,self.stderr = self.stdout.getvalue(),self.stderr.getvalue()
        send_record(self.wfile.write, (self.stdout,self.stderr))
        super().finish()
    def _handle(self, f):
        pool = getattr(self.server,'pool',None)
        if pool is not None: return pool.submit(f).result()
        return f()
    def handle(self):
        f = partial(_run, self.server.cmd, self.argv, self.stdin, self.stdout, self.stderr)
        self.stdout,self.stderr = self._handle(f)

`CmdHandler`'s primary use-case is together with a server inheriting `CmdMixin`, however it supports any `socketserver.BaseServer` that has a `cmd` attribute. If the server also has a `pool` it's used to execute `cmd`.

Here's an example of how to use `CmdHandler`. First, define the command. It should have no arguments itself but rather parse its arguments from `sys.argv`. Its return value isn't used, instead it should write to `stdout`.

In [ ]:
def _cmd():
    sys.stdout.write(sys.stdin.read()+sys.argv[1])
    sys.stderr.write('Error!')

Then define a server with a `cmd` attribute:

In [ ]:
class _CmdServer(UnixStreamServer): cmd = lambda x: _cmd()

Start the server. We start it with `handle_request` in a separate thread:

In [ ]:
p = Path('fdaemon.sock')
if p.exists(): p.unlink()

@threaded
def _f():
    with _CmdServer(str(p), CmdHandler) as srv: srv.handle_request()
_f()
time.sleep(0.2) # wait for server to start

...so that we can send a request and print its response:

In [ ]:
transfer(['Hello, ', 'world!'], str(p))

len(stdin)=7 argv=['<lambda>', 'world!']


['Hello, world!', 'Error!']

In [ ]:
#|export
class CmdMixin:
    "Socket server with a `cmd` and optional `pool`"
    def __init__(self, server_address, cmd, pool=None, RequestHandlerClass=CmdHandler, timeout=None, **kwargs):
        self.cmd,self.pool = cmd,pool
        if timeout is not None: self.timeout = timeout
        super().__init__(server_address, RequestHandlerClass)

    def handle_timeout(self): return True

In [ ]:
class CmdUnixServer(CmdMixin, ThreadingUnixStreamServer): pass

In [ ]:
p = Path('fdaemon.sock')
if p.exists(): p.unlink()

@threaded
def _f():
    with ProcessPoolExecutor() as pool, CmdUnixServer(str(p), _cmd, pool) as srv: srv.handle_request()
_f()
time.sleep(0.2) # wait for server to start

In [ ]:
transfer(['Hello, ', 'world!'], str(p))

len(stdin)=7 argv=['_cmd', 'world!']


['Hello, world!', 'Error!']

In [ ]:
#|export
class CmdTCPServer(CmdMixin, ThreadingTCPServer):
    allow_reuse_address = True

In [ ]:
#|export
def _fastdaemon_serve(cmd, port, host='localhost', timeout=None):
    "Serve `cmd` on `port`, with optional `host` and `timeout`"
    with ProcessPoolExecutor() as pool, CmdTCPServer((host,port), cmd, pool, timeout=timeout) as srv:
        while not srv.handle_request(): pass

A convenient wrapper to instantiate and start a `CmdTCPServer` that handles requests until it's interrupted or times out. Here's the previous example using `_fastdaemon_serve`. We've also set a `timeout` to avoid running forever:

In [ ]:
host,port = 'localhost',9999

@threaded
def _f(): _fastdaemon_serve(_cmd, port, host, timeout=1)
_f()
time.sleep(0.4) # wait for server to start

In [ ]:
transfer(['Hello, ', 'world!'], port, host)

len(stdin)=7 argv=['_cmd', 'world!']


['Hello, world!', 'Error!']

In [ ]:
#|export
def _import_cmd(cmd):
    mn, on = cmd.split(':')
    m = importlib.import_module(mn)
    return getattr(m,on)

In [ ]:
_import_cmd('nbprocess.clean:nbprocess_clean')

<function nbprocess.clean.nbprocess_clean(fname: str = None, clear_all: bool = False, disp: bool = False, stdin: bool = False)>

In [ ]:
#|export
@call_parse
def fastdaemon_serve(
    cmd:str, # Module path to callable command (example: pkg.mod:obj)
    port:int, # Server port
    host:str='localhost', # Server host
    timeout:int=None): # Shutdown after `timeout` seconds without requests
    "Serve `cmd` on `port`, with optional `host` and `timeout`"
    _cmd = _import_cmd(cmd)
    _fastdaemon_serve(_cmd, port, host, timeout) # TODO: dont need two functions because of call_parse

In [ ]:
!fastdaemon_serve -h

usage: fastdaemon_serve [-h] [--host HOST] [--timeout TIMEOUT] cmd port

Serve `cmd` on `port`, with optional `host` and `timeout`

positional arguments:
  cmd                Module path to callable command (example: pkg.mod:obj)
  port               Server port

optional arguments:
  -h, --help         show this help message and exit
  --host HOST        Server host (default: localhost)
  --timeout TIMEOUT  Shutdown after `timeout` seconds without requests


## Export -

In [ ]:
#|hide
#|eval: false
from nbprocess.doclinks import nbprocess_export
nbprocess_export()